In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [16]:
client_id = 'b05478021f3c4e73bbbd0482e81a3396'
client_secret = '535562f03be14a998dcd97448d5322e8'

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [36]:
# Chill Car - Rap, Bollywood Beats, Lit Songs, Fam Mix, Chill - Pop, Nostalgia, 
playlist_links = ["https://open.spotify.com/playlist/39yrrZvXXVTQjnv41mLZE5?si=043d2b062b4748c7",
                  "https://open.spotify.com/playlist/12p5ArAxY6r3QOtNATi7Mx?si=0e44c2c5c12d4f45",
                  "https://open.spotify.com/playlist/4KhumTd9QyJM3RUvAzSJh7?si=edc7bcc04f1c43a9",
                  "https://open.spotify.com/playlist/1OzHjXm9IwK7dlQk1mXP2N?si=05d4cd76d85f47df",
                  "https://open.spotify.com/playlist/2W4rSltujQSUyXT3Yu03ga?si=7ff0dd18a47b4517",
                  "https://open.spotify.com/playlist/4Xwamd29v8Lij8tsP9jslm?si=8ed5a2c219624e3c"]
playlist_URIs = [playlist_link.split("/")[-1].split("?")[0] for playlist_link in playlist_links]

In [42]:
song_data = []
seen_songs = set()
for playlist_URI in playlist_URIs:
    for track in sp.playlist_tracks(playlist_URI)["items"]:
        
        # URI
        track_uri = track["track"]["uri"]
        
        # Skip duplicate songs
        if track_uri in seen_songs:
            # Update the playlist URIs for the existing song
            existing_song_index = next((index for index, song in enumerate(song_data) if song[0] == track_uri), None)
            if existing_song_index is not None:
                song_data[existing_song_index][-1].add(playlist_URI)
            continue
        
        # Add song URI to seen_songs set
        seen_songs.add(track_uri)
        
        # Track name
        track_name = track["track"]["name"]
        
        # Main Artist
        artist_uri = track["track"]["artists"][0]["uri"]
        artist_info = sp.artist(artist_uri)
        
        # Name, popularity, genre
        artist_name = track["track"]["artists"][0]["name"]
        artist_pop = artist_info["popularity"]
        artist_genres = artist_info["genres"]
        
        # Album
        album = track["track"]["album"]["name"]
        
        # release date
        release_date = track["track"]["album"]["release_date"]

        # Popularity of the track
        track_pop = track["track"]["popularity"]
        
        # Audio Features
        audio_features = sp.audio_features(track_uri)[0]
        selected_features = {
            'danceability': audio_features['danceability'],
            'energy': audio_features['energy'],
            'key': audio_features['key'],
            'loudness': audio_features['loudness'],
            'speechiness': audio_features['speechiness'],
            'acousticness': audio_features['acousticness'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness'],
            'valence': audio_features['valence'],
            'tempo': audio_features['tempo']
        }
        
        song_data.append([track_uri, track_name, artist_name, artist_pop, artist_genres, album, track_pop, release_date] + list(selected_features.values()) + [{playlist_URI}])

# Create the dataframe
columns = ['Track URI', 'Track Name', 'Artist Name', 'Artist Popularity', 'Artist Genres', 'Album', 'Track Popularity', 'Release Date'] + list(selected_features.keys()) + ['Playlist URIs']
df = pd.DataFrame(song_data, columns=columns)

In [43]:
df

,Track URI,Track Name,Artist Name,Artist Popularity,Artist Genres,Album,Track Popularity,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,Playlist URIs
0,spotify:track:6NszDAUWi40uiQhXRT9p09,Shiva (with JID & EARTHGANG feat. Benji. & Jur...,Spillage Village,47,[underground hip hop],Spilligion,49,0.442,0.770,11,-4.369,0.5010,0.1090,0.000000,0.4150,0.341,192.179,{39yrrZvXXVTQjnv41mLZE5}
1,spotify:track:4D2Z8yUlhaDD81ubWpcRdt,All Over You,Majid Jordan,66,[canadian contemporary r&b],All Over You,1,0.797,0.500,0,-6.964,0.0354,0.0579,0.000000,0.1220,0.492,106.957,"{2W4rSltujQSUyXT3Yu03ga, 39yrrZvXXVTQjnv41mLZE5}"
2,spotify:track:2WxUIiq06XXPYWl9YcRJnD,Our Time,Lil Tecca,76,"[melodic rap, pluggnb, rap]",Virgo World,80,0.895,0.439,5,-12.142,0.3190,0.1990,0.000000,0.0683,0.633,109.988,{39yrrZvXXVTQjnv41mLZE5}
3,spotify:track:4mL25h2hl6gdyu9tNTudnI,Baby $hit,Freddie Gibbs,66,"[alternative hip hop, hip hop, indiana hip hop...",Alfredo,52,0.599,0.629,2,-11.325,0.3810,0.6650,0.016500,0.4470,0.577,130.346,{39yrrZvXXVTQjnv41mLZE5}
4,spotify:track:2Yer0p7uB2lVBUAtANuuQp,Magic In The Hamptons (feat. Lil Yachty),Social House,64,[],Magic In The Hamptons (feat. Lil Yachty),77,0.769,0.479,1,-5.339,0.0385,0.6600,0.000000,0.0914,0.803,96.051,"{2W4rSltujQSUyXT3Yu03ga, 39yrrZvXXVTQjnv41mLZE..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,spotify:track:0fYVliAYKHuPmECRs1pbRf,Renegades,X Ambassadors,71,"[modern alternative rock, modern rock, stomp pop]",VHS,80,0.526,0.862,2,-6.003,0.0905,0.0144,0.059700,0.2290,0.528,90.052,{4Xwamd29v8Lij8tsP9jslm}
503,spotify:track:2EBCVPNAG46nbgs6jXPGvv,Starships,Nicki Minaj,86,"[hip pop, pop, queens hip hop, rap]",Pink Friday: Roman Reloaded The Re-Up (Explici...,82,0.747,0.716,11,-2.457,0.0750,0.1350,0.000000,0.2510,0.751,125.008,{4Xwamd29v8Lij8tsP9jslm}
504,spotify:track:3hlksXnvbKogFdPbpO9vel,Super Bass,Nicki Minaj,86,"[hip pop, pop, queens hip hop, rap]",Pink Friday (Complete Edition),82,0.723,0.860,11,-4.357,0.2030,0.2710,0.000005,0.5980,0.667,126.979,{4Xwamd29v8Lij8tsP9jslm}
505,spotify:track:7LcfRTgAVTs5pQGEQgUEzN,"Moves Like Jagger - Studio Recording From ""The...",Maroon 5,84,[pop],Hands All Over,76,0.722,0.758,11,-4.477,0.0471,0.0111,0.000000,0.3080,0.620,128.047,{4Xwamd29v8Lij8tsP9jslm}
